# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kulhudhuffushi,6.6221,73.0700,28.50,73,73,5.82,MV,1722730706
1,1,ushuaia,-54.8000,-68.3000,-1.19,64,100,4.63,AR,1722731214
2,2,vadso,70.0744,29.7487,10.49,87,100,4.12,NO,1722731215
3,3,lautoka,-17.6167,177.4667,26.92,69,75,5.66,FJ,1722731216
4,4,fale old settlement,-9.3852,-171.2468,27.70,79,63,5.95,TK,1722731217


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot
map_city_data= city_data_df.hvplot.points(
    "Lng",
    "Lat",
     size = "Humidity",
     color= "City"
)
# Display the map
map_city_data

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df.loc[(city_data_df["Max Temp"] <= 27) & 
                                 (city_data_df["Max Temp"] >= 21) & 
                                 (city_data_df["Cloudiness"] == 0) & 
                                 (city_data_df["Wind Speed"] < 4.5)]

# Drop any rows with null values
clean_ideal_conditions = ideal_conditions_df.dropna()

# Display sample data
clean_ideal_conditions

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,jamestown,42.0970,-79.2353,23.80,80,0,1.54,US,1722730934
56,56,pacific grove,36.6177,-121.9166,26.79,86,0,3.60,US,1722731274
75,75,remire-montjoly,4.9167,-52.2667,25.02,94,0,1.03,GF,1722730782
77,77,messaad,34.1543,3.5031,26.76,32,0,3.56,DZ,1722731297
83,83,praia da vitoria,38.7333,-27.0667,23.20,83,0,1.62,PT,1722730895
84,84,bethel,41.3712,-73.4140,23.84,94,0,1.54,US,1722731305
97,97,dingli,35.8614,14.3822,26.09,83,0,2.57,MT,1722731319
130,130,martapura,-3.4167,114.8500,26.05,89,0,1.03,ID,1722731359
168,168,iyidere,41.0119,40.3619,23.98,80,0,1.86,TR,1722731405
222,222,peace river,56.2501,-117.2860,24.17,49,0,3.60,CA,1722730936


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df[["City", "Country", "Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kulhudhuffushi,MV,6.6221,73.0700,73,
1,ushuaia,AR,-54.8000,-68.3000,64,
2,vadso,NO,70.0744,29.7487,87,
3,lautoka,FJ,-17.6167,177.4667,69,
4,fale old settlement,TK,-9.3852,-171.2468,79,
...,...,...,...,...,...,...
587,magadan,RU,59.5667,150.8000,52,
588,borsa,RO,47.6553,24.6633,98,
589,rabaul,PG,-4.1967,152.1721,72,
590,aizawl,IN,23.7333,92.7167,98,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accomodation.hotel",
    "key": geoapify_key,
    "limit": 1 # first hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row["Lat"]
    longitude= row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address =name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kulhudhuffushi - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
vadso - nearest hotel: No hotel found
lautoka - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
geraldton - nearest hotel: No hotel found
ancud - nearest hotel: No hotel found
nemuro - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
merizo village - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
riviere-du-loup - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
puerto san jose - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
strzelin - nearest hotel: No hotel found
kiunga - nearest hotel: No hotel found
champerico - nearest hotel: No hotel found
murun-kuren - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
galle - nearest hotel: No hotel found

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kulhudhuffushi,MV,6.6221,73.0700,73,No hotel found
1,ushuaia,AR,-54.8000,-68.3000,64,No hotel found
2,vadso,NO,70.0744,29.7487,87,No hotel found
3,lautoka,FJ,-17.6167,177.4667,69,No hotel found
4,fale old settlement,TK,-9.3852,-171.2468,79,No hotel found
...,...,...,...,...,...,...
587,magadan,RU,59.5667,150.8000,52,No hotel found
588,borsa,RO,47.6553,24.6633,98,No hotel found
589,rabaul,PG,-4.1967,152.1721,72,No hotel found
590,aizawl,IN,23.7333,92.7167,98,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot
possible_destinations = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    color = "City",
    hover_cols = ["Country", "Hotel"]
)

# Display the map
possible_destinations

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country)